In [1]:
import tensorflow as tf
from tensorflow.keras import backend as K
import keras as k
from sklearn.model_selection import GridSearchCV
import sklearn
import pandas as pd
import numpy as np

# Dosya yolunu belirtin
file_path = 'a.xlsx'

# Verileri Excel'den okuma
Tmax  = pd.read_excel(file_path, usecols='AB', skiprows=0, nrows=4710)
Tmin  = pd.read_excel(file_path, usecols='AC', skiprows=0, nrows=4710)
RH  = pd.read_excel(file_path, usecols='AD', skiprows=0, nrows=4710)
Rs    = pd.read_excel(file_path, usecols='AE', skiprows=0, nrows=4710)
U2    = pd.read_excel(file_path, usecols='AF', skiprows=0, nrows=4710)
ETo   = pd.read_excel(file_path, usecols='AG', skiprows=0, nrows=4710)

# Verileri NumPy dizilerine dönüştürme
nTmax_num = Tmax.to_numpy().flatten()
nTmin_num = Tmin.to_numpy().flatten()
nRH_num = RH.to_numpy().flatten()
nRs_num = Rs.to_numpy().flatten()
nU2_num = U2.to_numpy().flatten()
nETO_num = ETo.to_numpy().flatten()

timesteps = 30
X = []
y = []

for i in range(len(nETO_num)):
    X.append([nTmax_num[i], nTmin_num[i], nRH_num[i], nRs_num[i], nU2_num[i]])
    y.append([nETO_num[i]])

X = np.array(X)
y = np.array(y)

print(X.shape)
print(y.shape)
X.shape[1]

(4708, 5)
(4708, 1)


5

In [2]:
import sklearn.model_selection


X = np.column_stack((nTmax_num, nTmin_num, nRH_num, nRs_num, nU2_num))
y = nETO_num

# Modeli oluşturma
model = k.models.Sequential()

# Giriş katmanı ve ilk gizli katmanı ekleme (10 nöronlu, ReLU aktivasyon fonksiyonu)
model.add(k.layers.Dense(16, activation='softmax'))

model.add(k.layers.Dense(8, activation='softmax'))
model.add(k.layers.Dense(4, activation='softmax'))

# Çıkış katmanı (Tek bir nöron, çünkü tahmin edilen değer tek bir skalar)
model.add(k.layers.Dense(1))

# Özel RMSE metriği
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# Modeli derleme
model.compile(optimizer='adam', loss='mean_squared_error',
              metrics=['mean_absolute_error', 'mean_squared_error', rmse])

# Modeli özetleme
model.summary()

# Eğitim ve doğrulama verilerini ayırma
X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(X, y, test_size=0.2)

early_stopping = k.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Modeli eğitme
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val))

# Eğitim verilerinde değerlendirme
train_loss, train_mae, train_mse, train_rmse = model.evaluate(X_train, y_train, verbose=0)

# Doğrulama verilerinde değerlendirme
val_loss, val_mae, val_mse, val_rmse = model.evaluate(X_val, y_val, verbose=0)

# Sonuçları yazdırma
print(f"Train Loss: {train_loss}")
print(f"Train MAE: {train_mae}")
print(f"Train MSE: {train_mse}")
print(f"Train RMSE: {train_rmse}")

print(f"Total Validation Loss (MSE): {val_loss}")
print(f"Validation Mean Absolute Error (MAE): {val_mae}")
print(f"Validation Mean Squared Error (MSE): {val_mse}")
print(f"Validation Root Mean Squared Error (RMSE): {val_rmse}")

# Tahmin yapma
predictions = model.predict(X)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 14.1419 - mean_absolute_error: 3.5455 - mean_squared_error: 14.1419 - rmse: 3.7529 - val_loss: 12.9861 - val_mean_absolute_error: 3.3902 - val_mean_squared_error: 12.9861 - val_rmse: 3.6070
Epoch 2/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 12.4545 - mean_absolute_error: 3.3010 - mean_squared_error: 12.4545 - rmse: 3.5211 - val_loss: 11.2808 - val_mean_absolute_error: 3.1294 - val_mean_squared_error: 11.2808 - val_rmse: 3.3615
Epoch 3/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 10.5772 - mean_absolute_error: 3.0127 - mean_squared_error: 10.5772 - rmse: 3.2442 - val_loss: 9.7641 - val_mean_absolute_error: 2.8785 - val_mean_squared_error: 9.7641 - val_rmse: 3.1269
Epoch 4/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 9.0753 - mean_absolute_error: 2.7513 - mean_squared_error: 9.0753 - rmse: 3.0056 - val_loss: 8.4297 - val_mean_absolute_error: 2.6395 - val_mean_squared_error: 8.4297 - val_rmse: 2.9049
Ep